In [1]:
from IPython.core.display import HTML
css = open('notebook_css/style-table.css').read() + open('notebook_css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Author: @imflash217 [September/06/2021]

# Quickatart

## Working with data

PyTorch has two primitives to work with data:

1. **`torch.utils.data.Dataset`**

2. **`torch.utils.data.DataLoader`**
   
**`Dataset`** stores the samples and their corresponding labels.

**`DataLoader`** wraps an **iterable** around the **`Dataset`**

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as `TorchText`, `TorchVision` & `TorchAudio`; all of which include datasets. Here, we will use `TorchVision` dataset.

The **`torchvision.datasets`** module contains `Dataset` objects for many real-world data like COCO, CIFAR etc.

Here we will use **FashionMNIST** dataset.

Every TorchVision **`Dataset`** object includes two arguments:

1. **`transform`**: modifies the samples

2. **`target_transform`**: modifies the labels

In [3]:
## 🎯 Download the data from open datasets

## TRAINING Data
training_data = datasets.FashionMNIST(root="data",
                                      train=True,
                                      download=True,
                                      transform=ToTensor(),
                                     )

## TEST data
test_data = datasets.FashionMNIST(root="data",
                                  train=False,
                                  download=True,
                                  transform=ToTensor(),
                                 )

We pass the `Dataset` as an argument to the `DataLoader`. 

This wraps an iterable over our dataset, and supports automatic **batching, sampling, shuffling & multiprocess-data-loading**.

In [5]:
## Here we define a batch size of 64
## (i.e. each element of the DataLoader iterable will return a batch of 64 features & labels)

batch_size = 64

## Create dataloaders
train_dataloader = DataLoader(dataset=training_data,
                              batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=batch_size)

for X, y in test_dataloader:
    print(f"X.shape [N,C,H,W] = {X.shape}")
    print(f"y.shape = {y.shape}, {y.dtype}")
    break

X.shape [N,C,H,W] = torch.Size([64, 1, 28, 28])
y.shape = torch.Size([64]), torch.int64


## Creating Models

To define a neural network in PyTorch, we create a class that inherits from **`torch.nn.Module`**. 

We define the layers of the network in the **`__init__()`** method and specify how data will pass through the network in the **`forward()`** method.

In [18]:
## Get cpu or gou device for training

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device.")

## Define Model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device=device)
print(model)
    

Using cpu device.
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model parameters

To train a model we need a **loss function** and an **optimizer**.

In [19]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-3)

In a single training loop, the model makes prediction on the training dataset (fed to it in batches), & backpropagates the prediction error to adjust the model's prediction

In [20]:
## the training loop

def train(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        
        ## Compute the prediction loss (aka. error)
        pred = model(X)
        loss = loss_func(pred, y)
        
        ## Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss = loss.item()
            current = batch * len(X)
            print(f"loss = {loss:>7f} | [{current:>5d}/{size:>5d}]")

In [21]:
## the test loop

def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy = {(100 * correct):>0.1f}% | Avg. Loss = {test_loss:>8f}\n")
            

In [22]:
## the training process

epochs = 5
for t in range(epochs):
    print(f"epoch = [{(t+1):2d}]\n{'---'*10}")
    
    train(dataloader=train_dataloader,
          model=model,
          loss_func=loss_func,
          optimizer=optimizer,)
    
    test(dataloader=test_dataloader, 
         model=model, 
         loss_func=loss_func,)

print("DONE!!🎯🎯")

epoch = [ 1]
------------------------------
loss = 2.301116 | [    0/60000]
loss = 2.293996 | [ 6400/60000]
loss = 2.270871 | [12800/60000]
loss = 2.266312 | [19200/60000]
loss = 2.246859 | [25600/60000]
loss = 2.215365 | [32000/60000]
loss = 2.220870 | [38400/60000]
loss = 2.189469 | [44800/60000]
loss = 2.187078 | [51200/60000]
loss = 2.149142 | [57600/60000]
Test Error: 
 Accuracy = 50.2% | Avg. Loss = 2.148793

epoch = [ 2]
------------------------------
loss = 2.155531 | [    0/60000]
loss = 2.151610 | [ 6400/60000]
loss = 2.090049 | [12800/60000]
loss = 2.103378 | [19200/60000]
loss = 2.048813 | [25600/60000]
loss = 1.990808 | [32000/60000]
loss = 2.006786 | [38400/60000]
loss = 1.935593 | [44800/60000]
loss = 1.934177 | [51200/60000]
loss = 1.849816 | [57600/60000]
Test Error: 
 Accuracy = 59.0% | Avg. Loss = 1.862372

epoch = [ 3]
------------------------------
loss = 1.889737 | [    0/60000]
loss = 1.870955 | [ 6400/60000]
loss = 1.751145 | [12800/60000]
loss = 1.784836 | [192

## Saving Models

A common way to save a model is to serialize the internal state dictionary (containing the model parameters)

In [26]:
torch.save(model.state_dict(), "models/model_1.pth")
print(f"Saved PyTorch model to 'models/model_1.pth'")

Saved PyTorch model to 'models/model_1.pth'


## Loading Models

The process of loading the models includes:

1. Re-creating the model structure / architecture

2. Then, loading the saved state-dictionary into it.

In [27]:
model = NeuralNetwork()
model.load_state_dict(torch.load("models/model_1.pth"))

<All keys matched successfully>

This model can now be used to make predictions

In [48]:
classes = ["T-shirt/top",
           "Trouser",
           "Pullover",
           "Dress",
           "Coat",
           "Sandal",
           "Shirt",
           "Sneaker",
           "Bag",
           "Ankle boot",
          ]

model.eval()
x = test_data[0][0]
y = test_data[0][1]

with torch.no_grad():
    pred = model(x)
    pred_label = classes[pred.argmax(dim=-1)]
    true_label = classes[y]
    print(f"Predicted_label = '{pred_label}' | True_label = '{true_label}'")
    

Predicted_label = 'Ankle boot' | True_label = 'Ankle boot'
